# Plot the extended SCvH EOS table

### Import modules

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import interpolate as interp
#from scipy import optimize
import numpy as np

In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'retina'

In [ ]:
mpl.rcParams['figure.dpi'] = 100

In [ ]:
def read_eos_table_dt(filename, delimiter=None):
    data = np.loadtxt(filename, delimiter=delimiter) 

    logT_table   = data[:, 0]
    logrho_table = data[:, 1]
    logP_table   = data[:, 2]
    logu_table   = data[:, 3]
    logs_table   = data[:, 4]

    # All the SCvH EOS tables are tabulated along isotherms
    logT = np.unique(logT_table)
    nT = np.size(logT)

    # For the mixture tables the number of grid points in rho the same for each isotherm
    nRho = int(np.size(logT_table)/nT)
    
    print("nT = {:} nRho = {:}".format(nT, nRho))
    
    logrho = logrho_table[0:nRho]
    
    logrho_min = np.min(logrho)
    logrho_max = np.max(logrho)
    logT_min   = np.min(logT)
    logT_max   = np.max(logT)

    print("logrho_min = {:}".format(logrho_min))
    print("logrho_max = {:}".format(logrho_max))
    print("logT_min   = {:}".format(logT_min))
    print("logT_max   = {:}".format(logT_max))
    print()

    # Split into arrays of constant T
    logP_array = np.split(logP_table, nT)
    logu_array = np.split(logu_table, nT)
    logs_array = np.split(logs_table, nT)

    # Generate 2d arrays
    logP = np.vstack(logP_array)
    logu = np.vstack(logu_array)
    logs = np.vstack(logs_array)
      
    eos_table_dt = {
        "nT":         nT,
        "nRho":       nRho,
        "logT":       logT,
        "logrho":     logrho,
        "logrho_min": logrho_min,
        "logrho_max": logrho_max,
        "logT_min":   logT_min,
        "logT_max":   logT_max,
        "logP":       logP,
        "logu":       logu,
        "logs":       logs,
    }
    
    return eos_table_dt

In [ ]:
# Read the eos table
scvh_hhe = read_eos_table_dt("scvh_extended_dt_hydrogen_722_helium_278.data")

In [ ]:
# Test if the eos table is read correctly
data = np.loadtxt("scvh_extended_dt_hydrogen_722_helium_278.data") 

logT_table   = data[:, 0]
logrho_table = data[:, 1]
logP_table   = data[:, 2]
logu_table   = data[:, 3]
logs_table   = data[:, 4]

# All the SCvH EOS tables are tabulated along isotherms
logT_axis = np.unique(logT_table)
nT = np.size(logT_axis)

print("Number of isotherms: nT = {:}".format(nT))

# The number of grid points in P are different for each isotherm
logrho_axis = list()
logP = list()
logu = list()
logs = list()

for logT in logT_axis:
    logrho_axis.append(logrho_table[np.where(logT_table == logT)])
    logP.append(logP_table[np.where(logT_table == logT)])
    logu.append(logu_table[np.where(logT_table == logT)])
    logs.append(logs_table[np.where(logT_table == logT)])

logT_min = np.min(logT_axis)
logT_max = np.max(logT_axis)

print("logT_min = {:}".format(logT_min))
print("logT_max = {:}".format(logT_max))
print()

nRho = list()
# Determine nRho
for i in range(nT):
    nRho.append(np.size(logrho_axis[i]))

nRho_min = np.min(nRho)
nRho_max = np.max(nRho)

print("nRho_min = {:}".format(nRho_min))
print("nRho_max = {:}".format(nRho_max))
print()

if nRho_min != nRho_max:
    print("Isotherms have different number of grid points: nRho_min={:} nRho_max={:}".format(nRho_min, nRho_max))
else:
    nRho = nRho_min

# Mark where the original SCvH EOS table was extended below T=100K
index = np.min(np.where(logT_axis >= 2.0))
print("The EOS table was extended until index={:} logT={:}".format(index, logT_axis[index]))

print(np.where(scvh_hhe['logT'] != logT_axis))

In [ ]:
# Skip some curves otherwise the plots are very hard to read
nSkipT = 4
nSkipRho = 10

In [ ]:
# Plot log(P), log(u) and log(s)
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# P(rho, T=const)
for i in range(0, scvh_hhe['nT'], nSkipT):
    ax[0][0].plot(scvh_hhe['logrho'], scvh_hhe['logP'][i,:], '-')
    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, scvh_hhe['nRho'], nSkipRho):
    ax[0][1].plot(scvh_hhe['logT'], scvh_hhe['logP'][:,i], '-')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, scvh_hhe['nT'], nSkipT):
    ax[1][0].plot(scvh_hhe['logrho'], scvh_hhe['logu'][i,:], '-')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, scvh_hhe['nRho'], nSkipRho):
    ax[1][1].plot(scvh_hhe['logT'], scvh_hhe['logu'][:,i], '-')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, scvh_hhe['nT'], nSkipT):
    ax[2][0].plot(scvh_hhe['logrho'], scvh_hhe['logs'][i,:], '-')

ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, scvh_hhe['nRho'], nSkipRho):
    ax[2][1].plot(scvh_hhe['logT'], scvh_hhe['logs'][:,i], '-')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
# Limit the data to the region in which we are interested
logrho_grid_min = scvh_hhe['logrho_min']
logrho_grid_max = -4.2

logT_grid_min = scvh_hhe['logT_min']
logT_grid_max = 3.5

# Remove the last isotherm because it causes problems when extrapolating
logT_grid_max = 3.46
logT_grid_max = 3.06
logT_grid_max = 3.3
# Compare to old table
logT_grid_max = 3.41


print("logrho_grid_min= {:} rho_grid_min= {:}".format(logrho_grid_min, 10**logrho_grid_min))
print("logrho_grid_max= {:} rho_grid_max= {:}".format(logrho_grid_max, 10**logrho_grid_max))
print("logT_grid_min= {:} T_grid_min= {:}".format(logT_grid_min, 10**logT_grid_min))
print("logT_grid_max= {:} T_grid_max= {:}".format(logT_grid_max, 10**logT_grid_max))
print()

index_logrho_grid = np.where(scvh_hhe['logrho'] < logrho_grid_max)[0]
index_logT_grid = np.where(scvh_hhe['logT'] < logT_grid_max)[0]

nRho_grid = np.size(index_logrho_grid)
nT_grid = np.size(index_logT_grid)
print("nRho= {:} nT= {:}".format(nRho_grid, nT_grid))

logrho_grid = scvh_hhe['logrho'][index_logrho_grid]
logT_grid = scvh_hhe['logT'][index_logT_grid]

logP_grid = scvh_hhe['logP'][:nT_grid,:nRho_grid]
logu_grid = scvh_hhe['logu'][:nT_grid,:nRho_grid]
logs_grid = scvh_hhe['logs'][:nT_grid,:nRho_grid]

print(scvh_hhe['logP'].shape)
print(logP_grid.shape)
#logP_grid = logP[index_logT_grid, index_logrho_grid]
#logu_grid = logu[index_logT_grid, index_logrho_grid]

print(scvh_hhe['logT'])

In [ ]:
# Plot log(P), log(u) and log(s) of the restricted grid
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# P(rho, T=const)
for i in range(0, nT_grid, nSkipT):
    ax[0][0].plot(logrho_grid, logP_grid[i,:], '-')
    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, nRho_grid, nSkipRho):
    ax[0][1].plot(logT_grid, logP_grid[:,i], '-')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, nT_grid, nSkipT):
    ax[1][0].plot(logrho_grid, logu_grid[i,:], '-')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, nRho_grid, nSkipRho):
    ax[1][1].plot(logT_grid, logu_grid[:,i], '-')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, nT_grid, nSkipT):
    ax[2][0].plot(logrho_grid, logs_grid[i,:], '-')

ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, nRho_grid, nSkipRho):
    ax[2][1].plot(logT_grid, logs_grid[:,i], '-')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
# Generate interpolation functions
logP_grid_int = interp.interp2d(logrho_grid, logT_grid, logP_grid, kind='cubic')
logu_grid_int = interp.interp2d(logrho_grid, logT_grid, logu_grid, kind='cubic')
logs_grid_int = interp.interp2d(logrho_grid, logT_grid, logs_grid, kind='cubic')

In [ ]:
# Test interpolation on the restricted grid
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# P(rho, T=const)
for i in range(0, nT_grid, nSkipT):
    ax[0][0].plot(logrho_grid, logP_grid[i,:], '-')
    ax[0][0].plot(logrho_grid, logP_grid_int(logrho_grid, logT_grid[i]), '--')

    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, nRho_grid, nSkipRho):
    ax[0][1].plot(logT_grid, logP_grid[:,i], '-')
    ax[0][1].plot(logT_grid, logP_grid_int(logrho_grid[i], logT_grid), '--')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, nT_grid, nSkipT):
    ax[1][0].plot(logrho_grid, logu_grid[i,:], '-')
    ax[1][0].plot(logrho_grid, logu_grid_int(logrho_grid, logT_grid[i]), '--')


ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, nRho_grid, nSkipRho):
    ax[1][1].plot(logT_grid, logu_grid[:,i], '-')
    ax[1][1].plot(logT_grid, logu_grid_int(logrho_grid[i], logT_grid), '--')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, nT_grid, nSkipT):
    ax[2][0].plot(logrho_grid, logs_grid[i,:], '-')
    ax[2][0].plot(logrho_grid, logs_grid_int(logrho_grid, logT_grid[i]), '--')


ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, nRho_grid, nSkipRho):
    ax[2][1].plot(logT_grid, logs_grid[:,i], '-')
    ax[2][1].plot(logT_grid, logs_grid_int(logrho_grid[i], logT_grid), '--')


ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
# Read the old eos table
scvh_hhe_old = read_eos_table_dt("eos-tables-old/scvh_hhe_y0.275_dt_cgs_lowrhot.csv", delimiter=",")

print("old table: logT={:}".format(scvh_hhe_old['logT']))
print("new table: logT={:}".format(logT_grid))


In [ ]:
# Plot log(P), log(u) and log(s) of the old and the new restricted table
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# P(rho, T=const)   
for i in range(0, scvh_hhe_old['nT'], nSkipT):
    ax[0][0].plot(scvh_hhe_old['logrho'], scvh_hhe_old['logP'][i,:], '-')
    ax[0][0].plot(scvh_hhe_old['logrho'], logP_grid_int(scvh_hhe_old['logrho'], scvh_hhe_old['logT'][i]), '--')

ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, scvh_hhe_old['nRho'], nSkipRho):
    ax[0][1].plot(scvh_hhe_old['logT'], scvh_hhe_old['logP'][:,i], '-')
    ax[0][1].plot(scvh_hhe_old['logT'], logP_grid_int(scvh_hhe_old['logrho'][i], scvh_hhe_old['logT']), '--')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, scvh_hhe_old['nT'], nSkipT):
    ax[1][0].plot(scvh_hhe_old['logrho'], scvh_hhe_old['logu'][i,:], '-')
    ax[1][0].plot(scvh_hhe_old['logrho'], logu_grid_int(scvh_hhe_old['logrho'], scvh_hhe_old['logT'][i]), '--')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, scvh_hhe_old['nRho'], nSkipRho):
    ax[1][1].plot(scvh_hhe_old['logT'], scvh_hhe_old['logu'][:,i], '-')
    ax[1][1].plot(scvh_hhe_old['logT'], logu_grid_int(scvh_hhe_old['logrho'][i], scvh_hhe_old['logT']), '--')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, scvh_hhe_old['nT'], nSkipT):
    ax[2][0].plot(scvh_hhe_old['logrho'], scvh_hhe_old['logs'][i,:], '-')
    ax[2][0].plot(scvh_hhe_old['logrho'], logs_grid_int(scvh_hhe_old['logrho'], scvh_hhe_old['logT'][i]), '--')

ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, scvh_hhe_old['nRho'], nSkipRho):
    ax[2][1].plot(scvh_hhe_old['logT'], scvh_hhe_old['logs'][:,i], '-')
    ax[2][1].plot(scvh_hhe_old['logT'], logs_grid_int(scvh_hhe_old['logrho'][i], scvh_hhe_old['logT']), '--')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
# Calculate the relative difference in pressure

err_rel_P = list()

for i in range(0, scvh_hhe_old['nT']):
    err = np.abs(scvh_hhe_old['logP'][i,:]-logP_grid_int(scvh_hhe_old['logrho'], scvh_hhe_old['logT'][i]))/scvh_hhe_old['logP'][i,:]
    err_rel_P.append(err)

err_rel_P = np.vstack(err_rel_P)


# Plot the difference
fig, ax = plt.subplots()

mark = np.zeros(err_rel_P.shape)

mark[np.where(err_rel_P >=0)] = 4
mark[np.where(err_rel_P < 1e-1)] = 3
mark[np.where(err_rel_P < 1e-3)] = 2
mark[np.where(err_rel_P < 1e-6)] = 1
mark[np.where(err_rel_P < 1e-10)] = 0

plt.imshow(mark, cmap=plt.cm.get_cmap('viridis', 5), origin='lower', interpolation='none', extent=[np.min(scvh_hhe_old['logrho']), np.max(scvh_hhe_old['logrho']), np.min(scvh_hhe_old['logT']), np.max(scvh_hhe_old['logT']) ], aspect='auto')
cbar = plt.colorbar()

plt.clim(-0.5, 4.5);

cbar.set_ticks(range(0, 5))
cbar.set_ticklabels(["$<10^{-10}$", "$<10^{-6}$", "$<10^{-3}$", "$<10^{-1}$", "$\geq 10^{-1}$"])

plt.xlabel("log(rho) [g cm^${-3}$]")
plt.ylabel("log(T) [K]")
plt.title("Pressure (relative error)")


In [ ]:
# Calculate the relative difference in internal energy

err_rel_u = list()

for i in range(0, scvh_hhe_old['nT']):
    err = np.abs(scvh_hhe_old['logu'][i,:]-logu_grid_int(scvh_hhe_old['logrho'], scvh_hhe_old['logT'][i]))/scvh_hhe_old['logu'][i,:]
    err_rel_u.append(err)

err_rel_u = np.vstack(err_rel_u)


# Plot the difference
fig, ax = plt.subplots()

mark = np.zeros(err_rel_u.shape)

mark[np.where(err_rel_u >=0)] = 4
mark[np.where(err_rel_u < 1e-1)] = 3
mark[np.where(err_rel_u < 1e-3)] = 2
mark[np.where(err_rel_u < 1e-6)] = 1
mark[np.where(err_rel_u < 1e-10)] = 0

plt.imshow(mark, cmap=plt.cm.get_cmap('viridis', 5), origin='lower', interpolation='none', extent=[np.min(scvh_hhe_old['logrho']), np.max(scvh_hhe_old['logrho']), np.min(scvh_hhe_old['logT']), np.max(scvh_hhe_old['logT']) ], aspect='auto')
cbar = plt.colorbar()

plt.clim(-0.5, 4.5);

cbar.set_ticks(range(0, 5))
cbar.set_ticklabels(["$<10^{-10}$", "$<10^{-6}$", "$<10^{-3}$", "$<10^{-1}$", "$\geq 10^{-1}$"])

plt.xlabel("log(rho) [g cm^${-3}$]")
plt.ylabel("log(T) [K]")
plt.title("Int. Energy (relative error)")


In [ ]:
# Calculate the relative difference in entropy

err_rel_s = list()

for i in range(0, scvh_hhe_old['nT']):
    err = np.abs(scvh_hhe_old['logs'][i,:]-logs_grid_int(scvh_hhe_old['logrho'], scvh_hhe_old['logT'][i]))/scvh_hhe_old['logs'][i,:]
    err_rel_s.append(err)

err_rel_s = np.vstack(err_rel_s)


# Plot the difference
fig, ax = plt.subplots()

mark = np.zeros(err_rel_s.shape)

mark[np.where(err_rel_s >=0)] = 4
mark[np.where(err_rel_s < 1e-1)] = 3
mark[np.where(err_rel_s < 1e-3)] = 2
mark[np.where(err_rel_s < 1e-6)] = 1
mark[np.where(err_rel_s < 1e-10)] = 0

plt.imshow(mark, cmap=plt.cm.get_cmap('viridis', 5), origin='lower', interpolation='none', extent=[np.min(scvh_hhe_old['logrho']), np.max(scvh_hhe_old['logrho']), np.min(scvh_hhe_old['logT']), np.max(scvh_hhe_old['logT']) ], aspect='auto')
cbar = plt.colorbar()

plt.clim(-0.5, 4.5);

cbar.set_ticks(range(0, 5))
cbar.set_ticklabels(["$<10^{-10}$", "$<10^{-6}$", "$<10^{-3}$", "$<10^{-1}$", "$\geq 10^{-1}$"])

plt.xlabel("log(rho) [g cm^${-3}$]")
plt.ylabel("log(T) [K]")
plt.title("Entropy (relative error)")


In [ ]:
def press_ideal_gas(rho, T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/(mu*m_H)*rho*T

In [ ]:
def intenergy_ideal_gas(T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/((gamma-1.0)*mu*m_H)*T

In [ ]:
def entropy_ideal_gas(T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/(mu*m_H)*(np.log(T/T0) - 1.0/(gamma-1.0)*np.log(rho/rho0))